                                                          Ф.И.О.

# Лабораторная работа 9

## Библиотека Scipy. Потоки

**1.** Выберите 3 разные функции (например, парабола, гипербола, прямая, коэффициенты подберите самостоятельно). Изобразите их графики на одной координатной плоскости разными цветами (добавьте легенду). Пользователь вводит номер функции и интересующий его интервал, программа отдельно отображает выбранную функцию на данном интервале и находит точку минимума функции на данном интервале. Кроме того, программа выводит значение интеграла функции между двумя введенными точками.

In [ ]:
import threading


**2.** Написать программу, которая вычисляет факториал числа 100 000 и показывает, сколько времени ушло на решение этой задачи. Сначала требуется решить задачу с использованием одного потока, затем с двумя, четырьмя, восьмью. При использовании двух потоков один поток перемножает числа от 1 до 50 000, а второй – от 50 001 до 100 000, а затем результаты перемножаются (можно поэкспериментировать с разделением работы на две части, не обязательно делить интервал пополам). Сравните результаты вычисления факториала (нужно, чтобы результаты совпадали) и время вычисления при каждом способе.

In [ ]:
import threading
import math
import time


# Функция для вычисления факториала на заданном интервале
def compute_factorial_range(start, end):
    result = 1
    for i in range(start, end + 1):
        result *= i
    return result


# Вычисление факториала с использованием разного количества потоков
def compute_factorial_with_threads(num_threads):
    # Вычисляем размер интервала для каждого потока
    interval_size = 100000 // num_threads
    threads = []
    # Разделяем интервал на части и создаем список потоков
    intervals = [(i * interval_size + 1, (i + 1) * interval_size) for i in range(num_threads)]
    results = []
    for start, end in intervals:
        # Создаем поток, который будет вычислять факториал на заданном интервале
        thread = threading.Thread(target=lambda s=start, e=end: results.append(compute_factorial_range(s, e)))
        threads.append(thread)
        thread.start()

    # Дожидаемся завершения всех потоков
    for thread in threads:
        thread.join()

    # Вычисляем итоговый результат путем перемножения результатов всех потоков
    total_result = 1
    for result in results:
        total_result *= result
    return total_result


if __name__ == '__main__':
    num_threads_list = [1, 2, 4, 8]
    for num_threads in num_threads_list:
        start_time = time.time()
        result = compute_factorial_with_threads(num_threads)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Вычисление факториала с {num_threads} потоками заняло {elapsed_time:.2f} секунд.')


**3.** Написать многопоточную программу, в которой каждый из двух потоков записывает в файл свой номер (1 или 2) 100 000 раз. Файл для обоих потоков один и тот же. Не допускается, чтобы сначала запись выполнял один поток полностью, а потом второй. Автоматически проверить, реально ли в результате в файле записано 100 000 единиц и 100 000 двоек.

In [4]:
import threading
import time


def check_writing_numbers(filename, count=200000):
    read_symbol_thread1 = int()
    read_symbol_thread2 = int()

    with open(filename, 'r') as f:
        read_symbol_thread1 = f.read().count('1')
    with open(filename, 'r') as f:
        read_symbol_thread2 = f.read().count('2')

    if read_symbol_thread2 == read_symbol_thread1 == count:
        print(f'Всё в порядке\nFILE:{filename}')
        return True
    else:
        print(
            f'Что-то не так. \nКоличество двоек:{read_symbol_thread2}\nКоличество единиц:{read_symbol_thread1}\nFILE:{filename}')
    return False


def write_no_sleep(filename, number):
    with open(filename, 'a') as f:
        for _ in range(200000):
            f.write(str(number))


def write_with_sleep(filename, number):
    with open(filename, 'a') as f:
        for _ in range(200000):
            f.write(str(number))
            time.sleep(0.0001)


def writing_without_sleep_thread(filename):
    print("WITH:")

    thread1 = threading.Thread(target=write_no_sleep, args=(filename, 1))
    thread2 = threading.Thread(target=write_no_sleep, args=(filename, 2))

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

    print("WITH END")


def writing_with_sleep_thread(filename):
    print("WITHOUT:")

    thread1 = threading.Thread(target=write_with_sleep, args=(filename, 1))
    thread2 = threading.Thread(target=write_with_sleep, args=(filename, 2))

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

    print("WITHOUT END")


filename = 'output_with_sleep.txt'
open(filename, 'w').close()

filename1 = 'output_without_sleep.txt'
open(filename1, 'w').close()

writing_without_sleep_thread(filename1)
check_writing_numbers(filename1)

writing_with_sleep_thread(filename)
check_writing_numbers(filename)



WITH:
WITH END
Что-то не так. 
Количество двоек:200000
Количество единиц:196608
FILE:output_without_sleep.txt
WITHOUT:
WITHOUT END
Всё в порядке
FILE:output_with_sleep.txt


True

**4.** Статистическая значимость. Изучите материалы, выданные Евгением Вячеславовичем на семинаре.

1) Сформируйте два датасета, в которые входят оценки качества двух методов:

- первый - содержит оценки качества для двух статистически неотличимых методов (то есть установите для обоих методов одинаковое среднее значение и сгенерируйте N оценок качества с заданным стандартным отклонением (отклонения можно установить разными для методов)),

- второй - то же самое, но средние методов должны отличаться.

2) Посчитайте статистическую значимость для уровней alpha = 0.05, 0.01, 0.001 для обоих датасетов по двухстороннему t-критерию Стьюдента. Выведите на графике распределения Стьюдента критические значения.

3) Посчитайте доверительные интервалы для обоих датасетов на уровне доверия 0.95.

In [53]:
import numpy as np
import pandas as pd
from scipy import stats

np.random.seed(10)

N = 10

mean_methos = 0.75

std_dev_method_first = 0.01
std_dev_method_second = 0.01

data_method1 = np.random.normal(loc=mean_methos, scale=std_dev_method_first, size=N)
data_method2 = np.random.normal(loc=mean_methos, scale=std_dev_method_second, size=N)

df_eq_mean = pd.DataFrame({
    'First Method': data_method1,
    'Second Method': data_method2
})
df_eq_mean['Difference'] = df_eq_mean['First Method'] - df_eq_mean['Second Method']

print(f"Неотличимые методы:\n{df_eq_mean}")

mean_method_third = 0.7
mean_method_fourth = 0.74

std_dev_method_third = 0.01
std_dev_method_fourth = 0.01

data_method1 = np.random.normal(loc=mean_method_third, scale=std_dev_method_third, size=N)
data_method2 = np.random.normal(loc=mean_method_fourth, scale=std_dev_method_fourth, size=N)

df_no_eq_mean = pd.DataFrame({
    'Third Method': data_method1,
    'Fourth Method': data_method2
})
df_no_eq_mean['Difference'] = df_no_eq_mean['Third Method'] - df_no_eq_mean['Fourth Method']

print(f"Отличимые методы:\n{df_no_eq_mean}")

print(
    f'mean1 = {df_eq_mean["First Method"].mean():.4f}, mean2 = {df_eq_mean["Second Method"].mean():.4f}, mean3 = {df_eq_mean["Difference"].mean():.4f}')
print(
    f'mean1 = {df_no_eq_mean["Third Method"].mean():.4f}, mean2 = {df_no_eq_mean["Fourth Method"].mean():.4f}, mean3 = {df_no_eq_mean["Difference"].mean():.4f}')

#################
# Рассчитываем t-статистику и p-значение для двух выборок
t_statistic_eq_mean, p_value_eq_mean = stats.ttest_ind(df_eq_mean['First Method'], df_eq_mean['Second Method'],
                                                       alternative='two-sided', equal_var=True)
t_statistic_no_eq_mean, p_value_no_eq_mean = stats.ttest_ind(df_no_eq_mean['Third Method'],
                                                             df_no_eq_mean['Fourth Method'], alternative='two-sided',
                                                             equal_var=True)

# Определите критические значения для разных уровней значимости
alpha_levels = [0.05, 0.01, 0.001]
critical_values = [stats.t.ppf(alpha / 2, N - 1) for alpha in alpha_levels]

# Вывод результатов
print(f"Для неотличимых методов:")
print(f"t-статистика: {t_statistic_eq_mean:.4f}, p-значение: {p_value_eq_mean:.4f}")
for alpha, critical_value in zip(alpha_levels, critical_values):
    if t_statistic_eq_mean > critical_value:
        print(
            f"На уровне значимости alpha={alpha}, t-статистика превышает критическое значение, отвергаем нулевую гипотезу.")
    else:
        print(
            f"На уровне значимости alpha={alpha}, t-статистика не превышает критическое значение, не отвергаем нулевую гипотезу.")

print(f"\nДля отличимых методов:")
print(f"t-статистика: {t_statistic_no_eq_mean:.4f}, p-значение: {p_value_no_eq_mean:.4f}")
for alpha, critical_value in zip(alpha_levels, critical_values):
    if t_statistic_no_eq_mean > critical_value:
        print(
            f"На уровне значимости alpha={alpha}, t-статистика превышает критическое значение, отвергаем нулевую гипотезу.")
    else:
        print(
            f"На уровне значимости alpha={alpha}, t-статистика не превышает критическое значение, не отвергаем нулевую гипотезу.")
##############

# Уровень доверия
confidence_level = 0.95

# Для первого датасета
data_method1 = df_eq_mean['Difference']  # Выбираем данные

# Рассчитываем доверительный интервал
confidence_interval_method1 = stats.t.interval(confidence_level, len(data_method1) - 1, loc=data_method1.mean(), scale=stats.sem(data_method1))

# Для второго датасета
data_method2 = df_no_eq_mean['Difference']  # Выбираем данные

# Рассчитываем доверительный интервал
confidence_interval_method2 = stats.t.interval(confidence_level, len(data_method2) - 1, loc=data_method2.mean(), scale=stats.sem(data_method2))

print(f"Доверительный интервал для первого датасета: {confidence_interval_method1}")
print(f"Доверительный интервал для второго датасета: {confidence_interval_method2}")

Неотличимые методы:
   First Method  Second Method  Difference
0      0.763316       0.754330    0.008986
1      0.757153       0.762030   -0.004878
2      0.734546       0.740349   -0.005803
3      0.749916       0.760283   -0.010367
4      0.756213       0.752286    0.003927
5      0.742799       0.754451   -0.011652
6      0.752655       0.738634    0.014021
7      0.751085       0.751351   -0.000266
8      0.750043       0.764845   -0.014802
9      0.748254       0.739202    0.009052
Отличимые методы:
   Third Method  Fourth Method  Difference
0      0.680223       0.737327   -0.057104
1      0.682566       0.734507   -0.051941
2      0.702661       0.741327   -0.038666
3      0.723850       0.735239   -0.011389
4      0.711237       0.753085   -0.041848
5      0.716726       0.741950   -0.025224
6      0.700991       0.744002   -0.043011
7      0.713980       0.736624   -0.022644
8      0.697288       0.752565   -0.055277
9      0.706132       0.732680   -0.026548
mean1 = 0.7506, 